In [1]:
# 导入必要的库
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import torch
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import torch.nn.functional as F
import nbimporter
from network import *
import os
import utils
from utils import *
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from graph import *
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import matplotlib

In [2]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# 读取制表符分隔的CSV文件
Methylation_train = pd.read_csv('../1_tr.csv',header=None)
# 显示 DataFrame
Methylation_train

,0,1,2,3,4,5,6,7,8,9,...,13723,13724,13725,13726,13727,13728,13729,13730,13731,13732
0,0.365973,0.217222,0.808067,0.460619,0.750717,0.949742,0.822088,0.870123,0.810249,0.292092,...,0.498429,0.740866,0.740866,0.957485,0.897986,0.815749,0.469954,0.364019,0.364019,0.675338
1,0.362246,0.015292,0.925907,0.308628,0.753873,0.924965,0.182585,0.818820,0.950813,0.933203,...,0.615475,0.869375,0.869375,0.829142,0.622149,0.852948,0.824037,0.606866,0.606866,0.757326
2,0.832305,0.090750,0.856745,0.484513,0.864601,0.376350,0.949966,0.661131,0.859642,0.746586,...,0.882364,0.833827,0.833827,0.825150,0.906963,0.855993,0.748098,0.713228,0.713228,0.233783
3,0.360447,0.424417,0.823998,0.452655,0.464286,0.991663,0.619786,0.943972,0.869932,0.811202,...,0.945994,0.953590,0.953590,0.831138,0.907448,0.854253,0.918357,0.669640,0.669640,0.145538
4,0.863531,0.027701,0.976482,0.118031,0.942197,0.916628,0.027126,0.904280,0.859024,0.816940,...,0.629222,0.891663,0.891663,0.895808,0.928675,0.954753,0.844447,0.673006,0.673006,0.653276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.425341,0.299448,0.624731,0.626106,0.750574,0.959371,0.588557,0.846092,0.819099,0.533571,...,0.667321,0.784314,0.784314,0.744511,0.849830,0.702197,0.912399,0.525076,0.525076,0.595654
362,0.345284,0.170970,0.891895,0.461173,0.888841,0.988492,0.398450,0.977184,0.937024,0.632388,...,0.927730,0.865425,0.865425,0.863273,0.874939,0.892756,0.970461,0.701784,0.701784,0.124794
363,0.682858,0.032088,0.875585,0.317367,0.947648,0.948567,0.465580,0.882274,0.931673,0.716516,...,0.397879,0.869093,0.869093,0.914172,0.872877,0.836633,0.807175,0.656345,0.656345,0.431347
364,0.691210,0.040612,0.930585,0.258739,0.952094,0.908408,0.246638,0.892129,0.959662,0.892574,...,0.721917,0.913246,0.913246,0.934930,0.929525,0.888841,0.913666,0.548132,0.548132,0.270662


In [4]:
# 读取制表符分隔的CSV文件
Methylation_test = pd.read_csv('../1_te.csv',header=None)
# 显示 DataFrame
Methylation_test

,0,1,2,3,4,5,6,7,8,9,...,13723,13724,13725,13726,13727,13728,13729,13730,13731,13732
0,0.900540,0.003886,0.976230,0.307522,0.956397,0.961837,0.768977,0.908735,0.959251,0.925973,...,0.724863,0.966709,0.966709,0.886228,0.553372,0.901457,0.471602,0.822955,0.822955,0.909121
1,0.314701,0.044623,0.893160,0.577323,0.803787,0.958079,0.782653,0.881193,0.922618,0.717319,...,0.748822,0.925801,0.925801,0.856287,0.914605,0.765499,0.892622,0.704981,0.704981,0.187685
2,0.796068,0.044247,0.888987,0.971018,0.952381,0.772076,0.956690,0.829486,0.848734,0.105475,...,0.579144,0.882917,0.882917,0.849900,0.911208,0.897324,0.271045,0.457253,0.457253,0.370102
3,0.416602,0.123088,0.816791,0.933518,0.793603,0.976984,0.558012,0.937357,0.848117,0.540342,...,0.892969,0.864438,0.864438,0.843912,0.915575,0.845116,0.943205,0.697240,0.697240,0.129075
4,0.524672,0.070820,0.643823,0.690708,0.831612,0.938469,0.922384,0.816120,0.749743,0.574544,...,0.516889,0.776414,0.776414,0.841717,0.876880,0.842288,0.496197,0.479132,0.479132,0.971024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.585068,0.027827,0.754331,0.951217,0.945783,0.722992,0.941418,0.590792,0.916032,0.161712,...,0.610369,0.718578,0.718578,0.802395,0.849224,0.862737,0.150101,0.501851,0.501851,0.730326
153,0.668209,0.412635,0.929195,0.554646,0.956684,0.974871,0.958286,0.877008,0.952459,0.795593,...,0.552435,0.880942,0.880942,0.880838,0.965915,0.733304,0.797667,0.679233,0.679233,0.596641
154,0.834104,0.014665,0.924516,0.981969,0.956540,0.272781,0.753590,0.488187,0.853674,0.068404,...,0.659073,0.636338,0.636338,0.863872,0.875425,0.835327,0.888692,0.561932,0.561932,0.135331
155,0.748522,0.062798,0.940448,0.987168,0.093230,0.763269,0.981422,0.905630,0.890307,0.209801,...,0.759819,0.745239,0.745239,0.859481,0.867419,0.936045,0.217039,0.568495,0.568495,0.164965


In [5]:
Methylation = pd.concat([Methylation_train, Methylation_test], axis=0)
Methylation

,0,1,2,3,4,5,6,7,8,9,...,13723,13724,13725,13726,13727,13728,13729,13730,13731,13732
0,0.365973,0.217222,0.808067,0.460619,0.750717,0.949742,0.822088,0.870123,0.810249,0.292092,...,0.498429,0.740866,0.740866,0.957485,0.897986,0.815749,0.469954,0.364019,0.364019,0.675338
1,0.362246,0.015292,0.925907,0.308628,0.753873,0.924965,0.182585,0.818820,0.950813,0.933203,...,0.615475,0.869375,0.869375,0.829142,0.622149,0.852948,0.824037,0.606866,0.606866,0.757326
2,0.832305,0.090750,0.856745,0.484513,0.864601,0.376350,0.949966,0.661131,0.859642,0.746586,...,0.882364,0.833827,0.833827,0.825150,0.906963,0.855993,0.748098,0.713228,0.713228,0.233783
3,0.360447,0.424417,0.823998,0.452655,0.464286,0.991663,0.619786,0.943972,0.869932,0.811202,...,0.945994,0.953590,0.953590,0.831138,0.907448,0.854253,0.918357,0.669640,0.669640,0.145538
4,0.863531,0.027701,0.976482,0.118031,0.942197,0.916628,0.027126,0.904280,0.859024,0.816940,...,0.629222,0.891663,0.891663,0.895808,0.928675,0.954753,0.844447,0.673006,0.673006,0.653276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.585068,0.027827,0.754331,0.951217,0.945783,0.722992,0.941418,0.590792,0.916032,0.161712,...,0.610369,0.718578,0.718578,0.802395,0.849224,0.862737,0.150101,0.501851,0.501851,0.730326
153,0.668209,0.412635,0.929195,0.554646,0.956684,0.974871,0.958286,0.877008,0.952459,0.795593,...,0.552435,0.880942,0.880942,0.880838,0.965915,0.733304,0.797667,0.679233,0.679233,0.596641
154,0.834104,0.014665,0.924516,0.981969,0.956540,0.272781,0.753590,0.488187,0.853674,0.068404,...,0.659073,0.636338,0.636338,0.863872,0.875425,0.835327,0.888692,0.561932,0.561932,0.135331
155,0.748522,0.062798,0.940448,0.987168,0.093230,0.763269,0.981422,0.905630,0.890307,0.209801,...,0.759819,0.745239,0.745239,0.859481,0.867419,0.936045,0.217039,0.568495,0.568495,0.164965


In [6]:
# 读取制表符分隔的CSV文件
SCNA_train = pd.read_csv('../2_tr.csv',header=None)
# 显示 DataFrame
SCNA_train

,0,1,2,3,4,5,6,7,8,9,...,18112,18113,18114,18115,18116,18117,18118,18119,18120,18121
0,0.691620,0.681759,0.161610,0.384683,0.069412,0.781085,0.813315,0.304145,0.0,0.582952,...,0.535518,0.432499,0.331222,0.667343,0.324827,0.650451,0.564977,0.508921,0.489148,0.000000
1,0.638114,0.269169,0.083599,0.444455,0.000000,0.471490,0.692427,0.830446,0.0,0.767858,...,0.329812,0.276381,0.830743,0.185368,0.341478,0.812707,0.780518,0.544684,0.574069,0.000000
2,0.351612,0.092728,0.000000,0.478671,0.118549,0.355224,0.604097,0.000000,0.0,0.540760,...,0.576340,0.587071,0.726864,0.074059,0.495038,0.568898,0.768283,0.770092,0.269943,0.000000
3,0.437127,0.803608,0.000000,0.410307,0.000000,0.770826,0.695892,0.716910,0.0,0.590196,...,0.768760,0.558428,0.705196,0.817918,0.496927,0.622598,0.702708,0.683873,0.475926,0.269879
4,0.766370,0.423349,0.126875,0.387531,0.000000,0.243778,0.694972,0.513566,0.0,1.000000,...,0.000000,0.165485,0.694321,0.543374,0.287168,0.711689,0.713528,0.499206,0.027598,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.454617,0.753309,0.000000,0.584099,0.000000,0.798638,0.731777,0.313639,0.0,0.511634,...,0.723876,0.564387,0.598900,0.615560,0.442793,0.730012,0.793948,0.547967,0.463478,0.000000
362,0.343075,0.888332,0.000000,0.630927,0.000000,0.736680,0.739514,0.559172,0.0,0.599128,...,0.973786,0.723006,0.696071,0.675459,0.647589,0.658306,0.692295,0.785552,0.475876,0.000000
363,0.489148,0.477032,0.059172,0.520571,0.055243,0.489162,0.799964,0.369995,0.0,0.674979,...,0.525553,0.358756,0.642527,0.079889,0.392337,0.835160,0.732804,0.537855,0.456139,0.000000
364,0.289832,0.260638,0.000000,0.535215,0.000000,0.271063,0.816205,0.559510,0.0,0.580222,...,0.509485,0.449077,0.649194,0.120440,0.496599,0.807956,0.795117,0.681967,0.838892,0.000000


In [7]:
# 读取制表符分隔的CSV文件
SCNA_test = pd.read_csv('../2_te.csv',header=None)
# 显示 DataFrame
SCNA_test

,0,1,2,3,4,5,6,7,8,9,...,18112,18113,18114,18115,18116,18117,18118,18119,18120,18121
0,0.396755,0.196299,0.513023,0.439814,0.047082,0.386658,0.773940,0.256985,0.000000,0.628444,...,0.545724,0.462516,0.742073,0.223491,0.489471,0.789373,0.737888,0.670796,0.762213,0.000000
1,0.464410,0.457391,0.052706,0.482048,0.000000,0.632977,0.756778,0.405364,0.000000,0.518327,...,0.579761,0.500846,0.565524,0.000000,0.397023,0.785041,0.668452,0.637151,0.597480,0.000000
2,0.320968,0.145329,0.728672,0.532333,0.072426,0.529656,0.290398,0.000000,0.000000,0.711842,...,0.651419,0.736052,0.764282,0.058941,0.602927,0.530373,0.686260,0.543784,0.287863,0.797142
3,0.422804,0.781739,0.161419,0.621795,0.000000,0.865667,0.713617,0.399260,0.147269,0.526974,...,0.781726,0.608824,0.667861,0.599014,0.563958,0.642538,0.743924,0.726149,0.606435,0.000000
4,0.573473,0.780522,0.466900,0.557636,0.000000,0.769391,0.635570,0.607587,0.132516,0.512240,...,0.613499,0.507271,0.581274,0.681446,0.479546,0.699848,0.751074,0.578754,0.732306,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.349440,0.000000,0.146052,0.319145,0.000000,0.524243,0.356118,0.000000,0.000000,0.676609,...,0.415216,0.634181,0.713655,0.000000,0.482997,0.489423,0.696727,0.443800,0.255426,0.000000
153,0.342824,0.539721,0.000000,0.464130,0.000000,0.515658,0.722441,0.361054,0.000000,0.645928,...,0.614980,0.426858,0.538564,0.665271,0.413522,0.703271,0.841634,0.611367,0.810069,0.000000
154,0.342638,0.000000,0.000000,0.412501,0.128054,0.648266,0.569492,0.000000,0.000000,0.673215,...,0.563253,0.622655,0.641318,0.000000,0.559827,0.448762,0.605731,0.439935,0.184881,0.000000
155,0.271436,0.000000,0.124504,0.292612,0.086931,0.503661,0.320093,0.000000,0.161893,0.670447,...,0.660744,0.715354,0.854619,0.125713,0.765051,0.469612,0.639144,0.454283,0.251290,0.000000


In [8]:
SCNA = pd.concat([SCNA_train, SCNA_test], axis=0)
SCNA

,0,1,2,3,4,5,6,7,8,9,...,18112,18113,18114,18115,18116,18117,18118,18119,18120,18121
0,0.691620,0.681759,0.161610,0.384683,0.069412,0.781085,0.813315,0.304145,0.000000,0.582952,...,0.535518,0.432499,0.331222,0.667343,0.324827,0.650451,0.564977,0.508921,0.489148,0.000000
1,0.638114,0.269169,0.083599,0.444455,0.000000,0.471490,0.692427,0.830446,0.000000,0.767858,...,0.329812,0.276381,0.830743,0.185368,0.341478,0.812707,0.780518,0.544684,0.574069,0.000000
2,0.351612,0.092728,0.000000,0.478671,0.118549,0.355224,0.604097,0.000000,0.000000,0.540760,...,0.576340,0.587071,0.726864,0.074059,0.495038,0.568898,0.768283,0.770092,0.269943,0.000000
3,0.437127,0.803608,0.000000,0.410307,0.000000,0.770826,0.695892,0.716910,0.000000,0.590196,...,0.768760,0.558428,0.705196,0.817918,0.496927,0.622598,0.702708,0.683873,0.475926,0.269879
4,0.766370,0.423349,0.126875,0.387531,0.000000,0.243778,0.694972,0.513566,0.000000,1.000000,...,0.000000,0.165485,0.694321,0.543374,0.287168,0.711689,0.713528,0.499206,0.027598,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.349440,0.000000,0.146052,0.319145,0.000000,0.524243,0.356118,0.000000,0.000000,0.676609,...,0.415216,0.634181,0.713655,0.000000,0.482997,0.489423,0.696727,0.443800,0.255426,0.000000
153,0.342824,0.539721,0.000000,0.464130,0.000000,0.515658,0.722441,0.361054,0.000000,0.645928,...,0.614980,0.426858,0.538564,0.665271,0.413522,0.703271,0.841634,0.611367,0.810069,0.000000
154,0.342638,0.000000,0.000000,0.412501,0.128054,0.648266,0.569492,0.000000,0.000000,0.673215,...,0.563253,0.622655,0.641318,0.000000,0.559827,0.448762,0.605731,0.439935,0.184881,0.000000
155,0.271436,0.000000,0.124504,0.292612,0.086931,0.503661,0.320093,0.000000,0.161893,0.670447,...,0.660744,0.715354,0.854619,0.125713,0.765051,0.469612,0.639144,0.454283,0.251290,0.000000


In [9]:
# 读取制表符分隔的CSV文件
RNAseq_train = pd.read_csv('../3_tr.csv',header=None)
# 显示 DataFrame
RNAseq_train

,0,1,2,3,4,5,6,7,8,9,...,507,508,509,510,511,512,513,514,515,516
0,0.612726,0.612321,0.616764,0.840863,0.824432,0.679596,0.584398,0.572609,0.519788,0.435956,...,0.067666,0.184775,0.406806,0.164561,0.366531,0.028418,0.353666,0.222934,0.575315,0.609072
1,0.349101,0.345472,0.341219,0.547733,0.823258,0.596723,0.391464,0.363991,0.389522,0.772756,...,0.000000,0.124275,0.000000,0.400721,0.394101,0.000000,0.220868,0.823515,0.378905,0.441094
2,0.171005,0.172425,0.176396,0.183163,0.106233,0.488475,0.592289,0.323834,0.383595,0.127185,...,0.024182,0.055034,0.286026,0.253897,0.245380,0.011862,0.195789,0.425371,0.463744,0.626723
3,0.747677,0.749276,0.749493,0.793992,0.840501,0.674479,0.664680,0.814957,0.860113,0.679458,...,0.000000,0.234543,0.071708,0.183106,0.489201,0.084565,0.395777,0.117711,0.695743,0.769608
4,0.379313,0.374070,0.372394,0.629942,0.859389,0.529045,0.178031,0.439774,0.385875,0.814161,...,0.030408,0.187829,0.119913,0.253838,0.240906,0.000000,0.232450,0.878420,0.347803,0.362682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.507499,0.503801,0.506340,0.838087,0.725779,0.579229,0.405096,0.566300,0.309844,0.177389,...,0.000000,0.239832,0.367393,0.105221,0.414526,0.183996,0.331185,0.080049,0.501470,0.671429
362,0.508912,0.510836,0.513857,0.817156,0.864401,0.846426,0.353009,0.542159,0.458964,0.385606,...,0.000000,0.162802,0.273945,0.290081,0.442339,0.071634,0.382241,0.469451,0.700976,0.791643
363,0.395017,0.395931,0.397350,0.638860,0.818418,0.545093,0.280790,0.446259,0.359178,0.512675,...,0.000000,0.257623,0.317185,0.411915,0.443615,0.052725,0.579749,0.672579,0.500862,0.352222
364,0.290605,0.293850,0.293714,0.551195,0.719906,0.294796,0.251692,0.433485,0.313606,0.177302,...,0.000000,0.131910,0.000000,0.064760,0.390923,0.000000,0.198562,0.425162,0.413534,0.302440


In [10]:
# 读取制表符分隔的CSV文件
RNAseq_test = pd.read_csv('../3_te.csv',header=None)
# 显示 DataFrame
RNAseq_test

,0,1,2,3,4,5,6,7,8,9,...,507,508,509,510,511,512,513,514,515,516
0,0.587404,0.588323,0.590050,0.650155,0.955270,0.470257,0.533532,0.656868,0.701623,0.718211,...,0.000000,0.302899,0.150205,0.381959,0.267505,0.000000,0.488671,0.763723,0.553189,0.353722
1,0.519644,0.521252,0.520806,0.687917,0.747424,0.438291,0.251887,0.409715,0.465211,0.505449,...,0.000000,0.368400,0.164798,0.283006,0.513694,0.052711,0.311047,0.412627,0.553443,0.260177
2,0.447155,0.449230,0.451751,0.785138,0.553046,0.574758,0.358823,0.244966,0.308431,0.448689,...,0.067141,0.104758,0.000000,0.093297,0.745531,0.112159,0.161959,0.747470,0.252793,0.576249
3,0.732218,0.731582,0.733081,0.732714,0.710023,0.598713,0.668980,0.854235,0.886239,0.507838,...,0.066574,0.215070,0.075865,0.191541,0.349325,0.063492,0.337064,0.145719,0.609474,0.687119
4,0.808754,0.807635,0.810437,0.919100,0.873015,0.595871,0.767023,0.605695,0.634051,0.549476,...,0.096911,0.069285,0.235729,0.174632,0.580575,0.092424,0.266553,0.280987,0.587250,0.621705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.487611,0.491789,0.488874,0.654159,0.469146,0.529666,0.406880,0.239634,0.440885,0.578526,...,0.000000,0.109073,0.288179,0.068288,0.296998,0.116271,0.096273,0.892574,0.339258,0.439479
153,0.440689,0.441601,0.442720,0.365863,0.588520,0.657812,0.637756,0.395516,0.644811,0.581323,...,0.000000,0.194800,0.618612,0.198341,0.541244,0.056580,0.473749,0.492466,0.598048,0.519757
154,0.434359,0.436298,0.435906,0.582863,0.514221,0.803323,0.566099,0.388914,0.551614,0.540938,...,0.080275,0.125249,0.253730,0.111547,0.450902,0.000000,0.110551,0.926324,0.381472,0.616186
155,0.615443,0.620302,0.618845,0.685173,0.259890,0.625459,0.642520,0.497510,0.647821,0.610373,...,0.000000,0.146227,0.302623,0.155239,0.375256,0.150465,0.129068,0.873682,0.295634,0.521026


In [11]:
RNAseq = pd.concat([RNAseq_train, RNAseq_test], axis=0)
RNAseq

,0,1,2,3,4,5,6,7,8,9,...,507,508,509,510,511,512,513,514,515,516
0,0.612726,0.612321,0.616764,0.840863,0.824432,0.679596,0.584398,0.572609,0.519788,0.435956,...,0.067666,0.184775,0.406806,0.164561,0.366531,0.028418,0.353666,0.222934,0.575315,0.609072
1,0.349101,0.345472,0.341219,0.547733,0.823258,0.596723,0.391464,0.363991,0.389522,0.772756,...,0.000000,0.124275,0.000000,0.400721,0.394101,0.000000,0.220868,0.823515,0.378905,0.441094
2,0.171005,0.172425,0.176396,0.183163,0.106233,0.488475,0.592289,0.323834,0.383595,0.127185,...,0.024182,0.055034,0.286026,0.253897,0.245380,0.011862,0.195789,0.425371,0.463744,0.626723
3,0.747677,0.749276,0.749493,0.793992,0.840501,0.674479,0.664680,0.814957,0.860113,0.679458,...,0.000000,0.234543,0.071708,0.183106,0.489201,0.084565,0.395777,0.117711,0.695743,0.769608
4,0.379313,0.374070,0.372394,0.629942,0.859389,0.529045,0.178031,0.439774,0.385875,0.814161,...,0.030408,0.187829,0.119913,0.253838,0.240906,0.000000,0.232450,0.878420,0.347803,0.362682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,0.487611,0.491789,0.488874,0.654159,0.469146,0.529666,0.406880,0.239634,0.440885,0.578526,...,0.000000,0.109073,0.288179,0.068288,0.296998,0.116271,0.096273,0.892574,0.339258,0.439479
153,0.440689,0.441601,0.442720,0.365863,0.588520,0.657812,0.637756,0.395516,0.644811,0.581323,...,0.000000,0.194800,0.618612,0.198341,0.541244,0.056580,0.473749,0.492466,0.598048,0.519757
154,0.434359,0.436298,0.435906,0.582863,0.514221,0.803323,0.566099,0.388914,0.551614,0.540938,...,0.080275,0.125249,0.253730,0.111547,0.450902,0.000000,0.110551,0.926324,0.381472,0.616186
155,0.615443,0.620302,0.618845,0.685173,0.259890,0.625459,0.642520,0.497510,0.647821,0.610373,...,0.000000,0.146227,0.302623,0.155239,0.375256,0.150465,0.129068,0.873682,0.295634,0.521026


In [12]:
Clinical_train = pd.read_csv('../labels_tr.csv', header=None)
Clinical_train

,0
0,1
1,2
2,2
3,1
4,2
...,...
361,1
362,1
363,2
364,2


In [13]:
Clinical_test = pd.read_csv('../labels_te.csv', header=None)
Clinical_test

,0
0,2
1,2
2,0
3,1
4,1
...,...
152,0
153,2
154,0
155,0


In [14]:
Clinical = pd.concat([Clinical_train, Clinical_test], axis=0)
Clinical

,0
0,1
1,2
2,2
3,1
4,2
...,...
152,0
153,2
154,0
155,0


In [15]:
X = Methylation
y = Clinical
Methylation_X_train, Methylation_X_test, Methylation_y_train, Methylation_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Methylation_X_train, Methylation_X_val, Methylation_y_train, Methylation_y_val = train_test_split(Methylation_X_train, Methylation_y_train, test_size=0.25, random_state=42)

In [16]:
Methylation_X_train.to_csv('../Methylation_train.csv', index=False)

In [17]:
Methylation_X_test.to_csv('../Methylation_test.csv', index=False)

In [18]:
Methylation_X_val.to_csv('../Methylation_val.csv', index=False)

In [19]:
Methylation_y_train.to_csv('../labels_train.csv', index=False)

In [20]:
Methylation_y_test.to_csv('../labels_test.csv', index=False)

In [21]:
Methylation_y_val.to_csv('../labels_val.csv', index=False)

In [22]:
X = SCNA
y = Clinical
SCNA_X_train, SCNA_X_test, SCNA_y_train, SCNA_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
SCNA_X_train, SCNA_X_val, SCNA_y_train, SCNA_y_val = train_test_split(SCNA_X_train, SCNA_y_train, test_size=0.25, random_state=42)

In [23]:
SCNA_X_train.to_csv('../SCNA_train.csv', index=False)

In [24]:
SCNA_X_test.to_csv('../SCNA_test.csv', index=False)

In [25]:
SCNA_X_val.to_csv('../SCNA_val.csv', index=False)

In [26]:
X = RNAseq
y = Clinical
RNAseq_X_train, RNAseq_X_test, RNAseq_y_train, RNAseq_y_test = train_test_split(X, y, test_size=0.2, random_state=42)
RNAseq_X_train, RNAseq_X_val, RNAseq_y_train, RNAseq_y_val = train_test_split(RNAseq_X_train, RNAseq_y_train, test_size=0.25, random_state=42)

In [27]:
RNAseq_X_train.to_csv('../RNAseq_train.csv', index=False)

In [28]:
RNAseq_X_test.to_csv('../RNAseq_test.csv', index=False)

In [29]:
RNAseq_X_val.to_csv('../RNAseq_val.csv', index=False)